## Imports

In [460]:
from pymongo import MongoClient
from pymongo.errors import BulkWriteError
import pymongo
client = MongoClient()
import sqlite3
import pandas as pd


## Config and Exploration

In [462]:
myclient = MongoClient()con = sqlite3.connect('rpg_db.sqlite3')
cur = sqlcon.cursor()
mydb = myclient['rpg']
char_record = mydb['character']
tables = [
        'charactercreator_character',
        'armory_item',
        'armory_weapon',
        'charactercreator_character_inventory',
        'charactercreator_cleric',
        'charactercreator_fighter',
        'charactercreator_mage',
        'charactercreator_thief',
        'charactercreator_necromancer'
    ]

In [518]:
def sql_df(query, con=con):
    return pd.read_sql(query, con)

sql_df('SELECT * FROM charactercreator_character LIMIT 10')

,character_id,name,level,exp,hp,strength,intelligence,dexterity,wisdom
0,1,Aliquid iste optio reiciendi,0,0,10,1,1,1,1
1,2,Optio dolorem ex a,0,0,10,1,1,1,1
2,3,Minus c,0,0,10,1,1,1,1
3,4,Sit ut repr,0,0,10,1,1,1,1
4,5,At id recusandae expl,0,0,10,1,1,1,1
5,6,Non nobis et of,0,0,10,1,1,1,1
6,7,Perferendis,0,0,10,1,1,1,1
7,8,Accusantium amet quidem eve,0,0,10,1,1,1,1
8,9,Sed nostrum inventore error m,0,0,10,1,1,1,1
9,10,Harum repellendus omnis od,0,0,10,1,1,1,1


## Insert Data to MongoDB

In [247]:
# index to prevent duplicates
result = char_record.create_index([('character_id', pymongo.ASCENDING)], unique=True)
characters = sql_df('SELECT * FROM charactercreator_character').to_dict(orient='records')
# Load data if not loaded already
try:
    result = char_record.insert_many(characters)
except BulkWriteError as e:
        print('Record already written')
    


In [248]:
# RPG Classes
base_classes = [
    'charactercreator_cleric',
    'charactercreator_fighter',
    'charactercreator_mage',
    'charactercreator_thief',
    'charactercreator_necromancer'
]
#merge the class data to character record
for cls in base_classes:
    for rec in sql_df(f'SELECT * FROM {cls}').to_dict(orient='records'):
        # Handle special case
        # Necrmonacer is a subclass of mage
        if cls == 'charactercreator_necromancer':
            character_id = rec.pop('mage_ptr_id')
        else:
            character_id = rec.pop('character_ptr_id')
        #use the last word for class name
        class_name = cls.split('_')[-1]
        rec[class_name] = True
        rec['items'] = []
        query = {'character_id': character_id}
        char_record.update_one(query, {'$set': rec})
    


In [519]:
# just double checking schema
sql_df('SELECT * FROM armory_item LIMIT 10')

,item_id,name,value,weight
0,1,"Libero facere dolore, as",0,0
1,2,Qui,0,0
2,3,Laborios,0,0
3,4,Quibusdam illo deserunt ea,0,0
4,5,Quod eveniet i,0,0
5,6,Qui odio beata,0,0
6,7,Omnis,0,0
7,8,Saepe ea vo,0,0
8,9,Vel illo sed,0,0
9,10,Pariatur hic iste m,0,0


In [520]:
# Add items
from math import isnan
for rec in sql_df('''
        SELECT character_id, ai.item_id, name, value, weight, power, weapon
        FROM charactercreator_character_inventory AS ci 
        JOIN armory_item AS ai ON ci.item_id=ai.item_id
        LEFT JOIN (SELECT *, True as weapon FROM armory_weapon) 
        AS aw on aw.item_ptr_id = ai.item_id
        ORDER BY ai.item_id
    ''').to_dict(orient='records'):
    character_id = rec.pop('character_id')
    keys = list(rec.keys())
    # Remove nan's as fields
    # joining the weapons introduced a lot of nan's to remove
    for key in keys:
        v = rec[key]     
        if type(v) == float and isnan(v):
            rec.pop(key)
    if rec.get('weapon') == 1:
        rec['weapon'] = True
    query = {'character_id': character_id}
    char_record.update_one(query, {'$addToSet': {'items': rec}})

In [521]:
#checking query
sql_df('''
 SELECT character_id, ai.item_id, name, value, weight, power, weapon
        FROM charactercreator_character_inventory AS ci 
        JOIN armory_item AS ai ON ci.item_id=ai.item_id
        JOIN (SELECT *, True as weapon FROM armory_weapon) 
        AS aw on aw.item_ptr_id = ai.item_id
        ORDER BY ai.item_id
''').head()

,character_id,item_id,name,value,weight,power,weapon
0,30,138,Corrupti sit at cum,0,0,0,1
1,128,138,Corrupti sit at cum,0,0,0,1
2,129,138,Corrupti sit at cum,0,0,0,1
3,70,139,Est fugit incidunt co,0,0,0,1
4,74,139,Est fugit incidunt co,0,0,0,1


## How many characters are there in total

In [464]:
char_record.count_documents({})

302

In [466]:
sql_df('SELECT COUNT(*) total_chars FROM charactercreator_character')

,total_chars
0,302


## How many characters per class

In [469]:
for cls in base_classes:
    cls_name = cls.split('_')[-1]
    cnt = char_record.count_documents({cls_name: True})
    print(f'{cls_name} count: {cnt}')

cleric count: 75
fighter count: 68
mage count: 108
thief count: 51
necromancer count: 11


In [474]:
sql_df('''
 SELECT 
        ( SELECT COUNT(*) FROM charactercreator_cleric ) AS cleric,
        ( SELECT COUNT(*) FROM charactercreator_fighter ) AS fighter,
        ( SELECT COUNT(*) FROM charactercreator_mage ) AS mage,
        ( SELECT COUNT(*) FROM charactercreator_thief ) AS thief,
        ( SELECT COUNT(*) FROM charactercreator_necromancer ) AS necromancer
''')

,cleric,fighter,mage,thief,necromancer
0,75,68,108,51,11


## Total Items

In [477]:
pipeline = [
    {'$unwind': '$items'},
    {
      '$count': "Total Items"
    }
]
list(char_record.aggregate(pipeline))

[{'Total Items': 898}]

In [484]:
sql_df('''
    SELECT count(*) AS total_items FROM armory_item AS ai
    JOIN charactercreator_character_inventory as cci
    on ai.item_id = cci.item_id
    
''')

,total_items
0,898


## How many of the Items are weapons? How many are not?'

In [524]:
pipeline = [
    {'$unwind': '$items'},
    {'$match': {'items.weapon': True}},
    {'$count': "Total weapons"}
]
list(char_record.aggregate(pipeline))

[{'Total weapons': 203}]

In [525]:
sql_df('''
    SELECT COUNT(*) AS wepons FROM armory_item as ai
    JOIN charactercreator_character_inventory as cci
    on ai.item_id = cci.item_id
    WHERE ai.item_id
    IN (SELECT item_ptr_id FROM armory_weapon)
''')

,wepons
0,203


In [526]:
pipeline = [
    {'$unwind': '$items'},
    {'$match': {'items.weapon': {'$ne':True}}},
    {'$count': "Total Non-weapons"}
]
list(char_record.aggregate(pipeline))

[{'Total Non-weapons': 695}]

In [494]:
sql_df('''
    SELECT COUNT(*) AS non_weapons FROM armory_item as ai
    JOIN charactercreator_character_inventory as cci
    on ai.item_id = cci.item_id
    WHERE ai.item_id
    NOT IN (SELECT item_ptr_id FROM armory_weapon)
''')

,non_weapons
0,695


## How many Items does each character have? (Return first 20 rows):

In [499]:
pipeline = [
    {'$project': {'_id': '$character_id', 'count': {'$size': '$items'}}},
    { '$limit': 20 }
]
list(char_record.aggregate(pipeline))

[{'_id': 1, 'count': 3},
 {'_id': 2, 'count': 3},
 {'_id': 3, 'count': 2},
 {'_id': 4, 'count': 4},
 {'_id': 5, 'count': 4},
 {'_id': 6, 'count': 1},
 {'_id': 7, 'count': 5},
 {'_id': 8, 'count': 3},
 {'_id': 9, 'count': 4},
 {'_id': 10, 'count': 4},
 {'_id': 11, 'count': 3},
 {'_id': 12, 'count': 3},
 {'_id': 13, 'count': 4},
 {'_id': 14, 'count': 4},
 {'_id': 15, 'count': 4},
 {'_id': 16, 'count': 1},
 {'_id': 17, 'count': 5},
 {'_id': 18, 'count': 5},
 {'_id': 19, 'count': 3},
 {'_id': 20, 'count': 1}]

In [501]:
sql_df('''
    SELECT character_id,
    count(charactercreator_character_inventory.item_id) as items
    FROM charactercreator_character_inventory
    GROUP by character_id LIMIT 20
''')

,character_id,items
0,1,3
1,2,3
2,3,2
3,4,4
4,5,4
5,6,1
6,7,5
7,8,3
8,9,4
9,10,4


## How many Weapons does each character have? (Return first 20 rows)

In [503]:
pipeline = [
    {'$unwind': '$items'},
    {'$project': {'character_id': 1, 'weapons': 
                  {'$cond': [ { '$eq': ["$items.weapon", True] }, 1, 0]}}},
    {'$group': {'_id': '$character_id', 'weapons': {'$sum':'$weapons'} }},
    {'$sort': {'_id':1}},
    { '$limit': 20 }
]
list(char_record.aggregate(pipeline))

[{'_id': 1, 'weapons': 0},
 {'_id': 2, 'weapons': 0},
 {'_id': 3, 'weapons': 0},
 {'_id': 4, 'weapons': 0},
 {'_id': 5, 'weapons': 2},
 {'_id': 6, 'weapons': 0},
 {'_id': 7, 'weapons': 1},
 {'_id': 8, 'weapons': 0},
 {'_id': 9, 'weapons': 0},
 {'_id': 10, 'weapons': 0},
 {'_id': 11, 'weapons': 1},
 {'_id': 12, 'weapons': 0},
 {'_id': 13, 'weapons': 0},
 {'_id': 14, 'weapons': 0},
 {'_id': 15, 'weapons': 0},
 {'_id': 16, 'weapons': 0},
 {'_id': 17, 'weapons': 0},
 {'_id': 18, 'weapons': 0},
 {'_id': 19, 'weapons': 0},
 {'_id': 20, 'weapons': 1}]

In [505]:
sql_df('''
    SELECT character_id, count(item_ptr_id) as weapons
    FROM charactercreator_character_inventory
    LEFT JOIN armory_weapon on
    charactercreator_character_inventory.item_id =
    armory_weapon.item_ptr_id
    GROUP by character_id LIMIT 20
''')

,character_id,weapons
0,1,0
1,2,0
2,3,0
3,4,0
4,5,2
5,6,0
6,7,1
7,8,0
8,9,0
9,10,0


## On average, how many Items does each Character have? 

In [514]:
#Average Items
pipeline = [
    {'$unwind': '$items'},
    {'$group': {'_id': '$character_id', 'count':{'$sum':1}}},
    {'$sort': {'_id':1}},
    {'$group': {'_id':'', 'Average Items':{'$avg':'$count'}}},
    
]
list(char_record.aggregate(pipeline))

[{'_id': '', 'Average Item': 2.9735099337748343}]

In [510]:
sql_df('''
    SELECT AVG(items) FROM
    (SELECT character_id,
    count(charactercreator_character_inventory.item_id)
    as items
    FROM charactercreator_character_inventory
    GROUP by character_id) AS FOO

''')

,AVG(items)
0,2.97351


## On average, how many Weapons does each Character have?

In [515]:
#Average Weapons
pipeline = [
    {'$unwind': '$items'},
    {'$project': {'character_id': 1, 'weapons': 
                  {'$cond': [ { '$eq': ["$items.weapon", True] }, 1, 0]}}},
    {'$group': {'_id': '$character_id', 'weapons': {'$sum':'$weapons'} }},
    {'$sort': {'_id':1}},
    {'$group': {'_id':'', 'Average Weapons':{'$avg':'$weapons'}}},
    
]
list(char_record.aggregate(pipeline))

[{'_id': '', 'Average Weapons': 0.6721854304635762}]

In [517]:
sql_df('''
    SELECT AVG(weapons) FROM
    (SELECT character_id, count(item_ptr_id) as weapons
    FROM charactercreator_character_inventory
    LEFT JOIN armory_weapon on
    charactercreator_character_inventory.item_id =
    armory_weapon.item_ptr_id
    GROUP by character_id) AS FOO
''')

,AVG(weapons)
0,0.672185


In [495]:
pipeline = [
    {'$unwind': '$items'},
    {'$match': {'items.weapon': {'$ne':True}}},
    {'$count': "Total Non-weapons"}
]
list(char_record.aggregate(pipeline))

[{'Total Non-weapons': 695}]

In [396]:
pipeline = [
    {'$project': {'character_id': '$character_id', 'count': {'$size': '$items'}}},
    { '$limit': 20 }
]
list(char_record.aggregate(pipeline))

[{'_id': ObjectId('5d2fd7eb3f66593262d7c988'), 'character_id': 1, 'count': 3},
 {'_id': ObjectId('5d2fd7eb3f66593262d7c989'), 'character_id': 2, 'count': 3},
 {'_id': ObjectId('5d2fd7eb3f66593262d7c98a'), 'character_id': 3, 'count': 2},
 {'_id': ObjectId('5d2fd7eb3f66593262d7c98b'), 'character_id': 4, 'count': 4},
 {'_id': ObjectId('5d2fd7eb3f66593262d7c98c'), 'character_id': 5, 'count': 4},
 {'_id': ObjectId('5d2fd7eb3f66593262d7c98d'), 'character_id': 6, 'count': 1},
 {'_id': ObjectId('5d2fd7eb3f66593262d7c98e'), 'character_id': 7, 'count': 5},
 {'_id': ObjectId('5d2fd7eb3f66593262d7c98f'), 'character_id': 8, 'count': 3},
 {'_id': ObjectId('5d2fd7eb3f66593262d7c990'), 'character_id': 9, 'count': 4},
 {'_id': ObjectId('5d2fd7eb3f66593262d7c991'), 'character_id': 10, 'count': 4},
 {'_id': ObjectId('5d2fd7eb3f66593262d7c992'), 'character_id': 11, 'count': 3},
 {'_id': ObjectId('5d2fd7eb3f66593262d7c993'), 'character_id': 12, 'count': 3},
 {'_id': ObjectId('5d2fd7eb3f66593262d7c994'), 'c

In [449]:
pipeline = [
    {'$unwind': '$items'},
    {'$group': {'_id': '$character_id', 'count':{'$sum':1}}},
    {'$sort': {'_id':1}},
    { '$limit': 20 }
    
]
list(char_record.aggregate(pipeline))

[{'_id': 1, 'count': 3},
 {'_id': 2, 'count': 3},
 {'_id': 3, 'count': 2},
 {'_id': 4, 'count': 4},
 {'_id': 5, 'count': 4},
 {'_id': 6, 'count': 1},
 {'_id': 7, 'count': 5},
 {'_id': 8, 'count': 3},
 {'_id': 9, 'count': 4},
 {'_id': 10, 'count': 4},
 {'_id': 11, 'count': 3},
 {'_id': 12, 'count': 3},
 {'_id': 13, 'count': 4},
 {'_id': 14, 'count': 4},
 {'_id': 15, 'count': 4},
 {'_id': 16, 'count': 1},
 {'_id': 17, 'count': 5},
 {'_id': 18, 'count': 5},
 {'_id': 19, 'count': 3},
 {'_id': 20, 'count': 1}]

In [451]:
pipeline = [
    {'$unwind': '$items'},
    {'$project': {'character_id': 1, 'weapons': 
                  {'$cond': [ { '$eq': ["$items.weapon", True] }, 1, 0]}}},
    {'$group': {'_id': '$character_id', 'weapons': {'$sum':'$weapons'} }},
    {'$sort': {'_id':1}},
    { '$limit': 20 }
]
list(char_record.aggregate(pipeline))

[{'_id': 1, 'weapons': 0},
 {'_id': 2, 'weapons': 0},
 {'_id': 3, 'weapons': 0},
 {'_id': 4, 'weapons': 0},
 {'_id': 5, 'weapons': 2},
 {'_id': 6, 'weapons': 0},
 {'_id': 7, 'weapons': 1},
 {'_id': 8, 'weapons': 0},
 {'_id': 9, 'weapons': 0},
 {'_id': 10, 'weapons': 0},
 {'_id': 11, 'weapons': 1},
 {'_id': 12, 'weapons': 0},
 {'_id': 13, 'weapons': 0},
 {'_id': 14, 'weapons': 0},
 {'_id': 15, 'weapons': 0},
 {'_id': 16, 'weapons': 0},
 {'_id': 17, 'weapons': 0},
 {'_id': 18, 'weapons': 0},
 {'_id': 19, 'weapons': 0},
 {'_id': 20, 'weapons': 1}]

In [456]:
#Average Items
pipeline = [
    {'$unwind': '$items'},
    {'$group': {'_id': '$character_id', 'count':{'$sum':1}}},
    {'$sort': {'_id':1}},
    {'$group': {'_id':'', 'Average':{'$avg':'$count'}}},
    
]
list(char_record.aggregate(pipeline))

[{'_id': '', 'Average': 2.9735099337748343}]

In [458]:
#Average Weapons
pipeline = [
    {'$unwind': '$items'},
    {'$project': {'character_id': 1, 'weapons': 
                  {'$cond': [ { '$eq': ["$items.weapon", True] }, 1, 0]}}},
    {'$group': {'_id': '$character_id', 'weapons': {'$sum':'$weapons'} }},
    {'$sort': {'_id':1}},
    {'$group': {'_id':'', 'Average':{'$avg':'$weapons'}}},
    
]
list(char_record.aggregate(pipeline))

[{'_id': '', 'Average': 0.6721854304635762}]